# QuantumEdge Tutorial 4: Advanced Optimization Objectives

**Deep Dive into CVaR, Calmar, and Sortino Ratio Optimization**

This notebook provides an in-depth exploration of advanced portfolio optimization objectives beyond traditional mean-variance optimization. We'll examine how different risk measures lead to varying portfolio characteristics and performance outcomes.

## Advanced Objectives Covered

- **CVaR (Conditional Value at Risk)**: Minimizes tail risk by focusing on worst-case scenarios
- **Sortino Ratio**: Maximizes return relative to downside deviation (only negative volatility)
- **Calmar Ratio**: Maximizes return relative to maximum drawdown
- **Comparative Analysis**: How these objectives differ from traditional Sharpe ratio optimization

---

## 1. Setup and Theoretical Background

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import time
from datetime import datetime, timedelta
import yfinance as yf
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (12, 8)

print("QuantumEdge Advanced Objectives Analysis")
print("=" * 45)
print("Exploring CVaR, Sortino, and Calmar Ratio Optimization")

In [ ]:
# Display theoretical background
print("\n📚 THEORETICAL BACKGROUND")
print("=" * 30)

backgrounds = {
    "CVaR (Conditional Value at Risk)": [
        "• Measures expected loss in the worst α% of cases",
        "• Also known as Expected Shortfall (ES)",
        "• More conservative than VaR as it considers tail distribution",
        "• Formula: CVaR_α = E[L | L ≥ VaR_α]",
        "• Useful for risk management and regulatory compliance"
    ],
    "Sortino Ratio": [
        "• Modified Sharpe ratio using only downside deviation",
        "• Distinguishes between 'good' and 'bad' volatility",
        "• Formula: (Return - Risk-free rate) / Downside deviation",
        "• Better for asymmetric return distributions",
        "• Preferred by investors focused on downside protection"
    ],
    "Calmar Ratio": [
        "• Return per unit of maximum drawdown",
        "• Formula: CAGR / |Maximum Drawdown|",
        "• Focuses on worst peak-to-trough decline",
        "• Important for capital preservation strategies",
        "• Commonly used in hedge fund analysis"
    ]
}

for objective, details in backgrounds.items():
    print(f"\n{objective}:")
    for detail in details:
        print(f"  {detail}")

In [ ]:
# QuantumEdge API configuration
API_BASE_URL = "http://localhost:8000/api/v1"

def check_api_health():
    """Check if QuantumEdge API is running"""
    try:
        response = requests.get(f"{API_BASE_URL.replace('/api/v1', '')}/health")
        if response.status_code == 200:
            print("✅ QuantumEdge API is healthy")
            return True
        else:
            print(f"❌ API health check failed: {response.status_code}")
            return False
    except Exception as e:
        print(f"❌ Cannot connect to API: {e}")
        return False

api_healthy = check_api_health()

## 2. Data Preparation and Market Analysis

We'll use a diverse set of assets including different risk profiles to highlight the differences between optimization objectives.

In [ ]:
# Define asset universe with different risk characteristics
ASSETS = {
    # Growth stocks (high volatility)
    'TSLA': 'Tesla Inc',
    'NVDA': 'NVIDIA Corp',
    'AMZN': 'Amazon.com Inc',
    
    # Stable large-caps
    'AAPL': 'Apple Inc',
    'MSFT': 'Microsoft Corp',
    'JNJ': 'Johnson & Johnson',
    
    # Financial sector (cyclical)
    'JPM': 'JPMorgan Chase',
    'BRK-B': 'Berkshire Hathaway'
}

SYMBOLS = list(ASSETS.keys())
START_DATE = '2020-01-01'
END_DATE = '2024-01-01'

print(f"Asset Universe ({len(SYMBOLS)} assets):")
print("=" * 35)
for symbol, name in ASSETS.items():
    print(f"{symbol:8} - {name}")

print(f"\nAnalysis Period: {START_DATE} to {END_DATE}")

In [ ]:
# Download and process market data
print("Downloading market data...")
data = yf.download(SYMBOLS, start=START_DATE, end=END_DATE, progress=False)['Adj Close']
returns = data.pct_change().dropna()

# Calculate key statistics
annual_returns = returns.mean() * 252
annual_volatility = returns.std() * np.sqrt(252)
sharpe_ratios = annual_returns / annual_volatility

# Calculate downside statistics
risk_free_rate = 0.02
downside_returns = returns[returns < risk_free_rate/252]  # Daily risk-free rate
downside_volatility = returns[returns < 0].std() * np.sqrt(252)  # Downside vol
sortino_ratios = (annual_returns - risk_free_rate) / downside_volatility

# Calculate maximum drawdown for each asset
def calculate_max_drawdown(price_series):
    """Calculate maximum drawdown for a price series"""
    cumulative = (1 + price_series.pct_change()).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    return drawdown.min()

max_drawdowns = data.apply(calculate_max_drawdown)
calmar_ratios = annual_returns / abs(max_drawdowns)

# Create summary statistics DataFrame
stats_df = pd.DataFrame({
    'Symbol': SYMBOLS,
    'Annual Return': annual_returns,
    'Annual Volatility': annual_volatility,
    'Sharpe Ratio': sharpe_ratios,
    'Downside Volatility': downside_volatility,
    'Sortino Ratio': sortino_ratios,
    'Max Drawdown': max_drawdowns,
    'Calmar Ratio': calmar_ratios
}).round(4)

print("\nAsset Statistics Summary:")
print("=" * 30)
print(stats_df.to_string(index=False))

In [ ]:
# Visualize asset characteristics
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Risk-Return Profile
axes[0,0].scatter(annual_volatility, annual_returns, s=100, alpha=0.7)
for i, symbol in enumerate(SYMBOLS):
    axes[0,0].annotate(symbol, (annual_volatility.iloc[i], annual_returns.iloc[i]),
                      xytext=(5, 5), textcoords='offset points')
axes[0,0].set_xlabel('Annual Volatility')
axes[0,0].set_ylabel('Annual Return')
axes[0,0].set_title('Risk-Return Profile')
axes[0,0].grid(True, alpha=0.3)

# 2. Sharpe vs Sortino Ratios
axes[0,1].scatter(sharpe_ratios, sortino_ratios, s=100, alpha=0.7, color='green')
for i, symbol in enumerate(SYMBOLS):
    axes[0,1].annotate(symbol, (sharpe_ratios.iloc[i], sortino_ratios.iloc[i]),
                      xytext=(5, 5), textcoords='offset points')
axes[0,1].set_xlabel('Sharpe Ratio')
axes[0,1].set_ylabel('Sortino Ratio')
axes[0,1].set_title('Sharpe vs Sortino Ratios')
axes[0,1].grid(True, alpha=0.3)

# Add diagonal line
min_val = min(sharpe_ratios.min(), sortino_ratios.min())
max_val = max(sharpe_ratios.max(), sortino_ratios.max())
axes[0,1].plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.5, label='Equal ratios')
axes[0,1].legend()

# 3. Maximum Drawdown Analysis
max_dd_abs = abs(max_drawdowns)
bars = axes[1,0].bar(SYMBOLS, max_dd_abs, alpha=0.7, color='red')
axes[1,0].set_title('Maximum Drawdown by Asset')
axes[1,0].set_ylabel('Maximum Drawdown')
axes[1,0].tick_params(axis='x', rotation=45)
axes[1,0].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1%}'))
axes[1,0].grid(True, alpha=0.3)

# Add value labels
for bar, value in zip(bars, max_dd_abs):
    axes[1,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                  f'{value:.1%}', ha='center', va='bottom')

# 4. Calmar Ratio Comparison
calmar_finite = calmar_ratios[np.isfinite(calmar_ratios)]  # Remove infinite values
bars = axes[1,1].bar(calmar_finite.index, calmar_finite.values, alpha=0.7, color='purple')
axes[1,1].set_title('Calmar Ratio by Asset')
axes[1,1].set_ylabel('Calmar Ratio')
axes[1,1].tick_params(axis='x', rotation=45)
axes[1,1].grid(True, alpha=0.3)

# Add value labels
for bar, value in zip(bars, calmar_finite.values):
    axes[1,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                  f'{value:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 3. CVaR Optimization Analysis

CVaR (Conditional Value at Risk) optimization focuses on minimizing the expected loss in the worst-case scenarios.

In [ ]:
def optimize_portfolio(objective, **kwargs):
    """Optimize portfolio using QuantumEdge API"""
    
    # Prepare base request
    payload = {
        "expected_returns": annual_returns.tolist(),
        "covariance_matrix": (returns.cov() * 252).values.tolist(),  # Annualized covariance
        "objective": objective,
        "risk_aversion": 1.0,
        **kwargs
    }
    
    # Add historical returns for advanced objectives
    if objective in ['minimize_cvar', 'maximize_sortino', 'maximize_calmar']:
        payload["returns_data"] = returns.values.tolist()
        
        if objective == 'minimize_cvar':
            payload["cvar_confidence"] = kwargs.get('cvar_confidence', 0.05)
        elif objective == 'maximize_calmar':
            payload["lookback_periods"] = kwargs.get('lookback_periods', 252)
    
    try:
        response = requests.post(f"{API_BASE_URL}/optimize/mean-variance", json=payload)
        
        if response.status_code == 200:
            result = response.json()
            if result['success']:
                return result
            else:
                print(f"❌ Optimization failed: {result.get('message', 'Unknown error')}")
                return None
        else:
            print(f"❌ API request failed: {response.status_code}")
            return None
            
    except Exception as e:
        print(f"❌ Error in optimization: {e}")
        return None

In [ ]:
# CVaR Sensitivity Analysis
if api_healthy:
    print("🔍 CVaR Sensitivity Analysis")
    print("=" * 30)
    
    cvar_levels = [0.01, 0.05, 0.10, 0.15]  # 1%, 5%, 10%, 15% confidence levels
    cvar_results = {}
    
    for confidence in cvar_levels:
        print(f"\nOptimizing CVaR at {confidence*100:.0f}% confidence level...")
        result = optimize_portfolio('minimize_cvar', cvar_confidence=confidence)
        
        if result:
            cvar_results[confidence] = result
            portfolio = result['portfolio']
            print(f"✅ Success - Expected Return: {portfolio['expected_return']:.4f}")
            print(f"          Volatility: {portfolio['volatility']:.4f}")
            print(f"          Sharpe Ratio: {portfolio['sharpe_ratio']:.3f}")
        
        time.sleep(1)  # Rate limiting
    
    print(f"\n📊 Completed {len(cvar_results)} CVaR optimizations")
else:
    print("❌ API not available - skipping CVaR analysis")
    cvar_results = {}

In [ ]:
# Analyze CVaR portfolio characteristics
if cvar_results:
    # Create CVaR analysis DataFrame
    cvar_analysis = []
    
    for confidence, result in cvar_results.items():
        portfolio = result['portfolio']
        weights = np.array(portfolio['weights'])
        
        # Calculate portfolio metrics
        num_assets = np.sum(weights > 0.01)
        max_weight = np.max(weights)
        diversification = 1 / np.sum(weights**2)  # Effective number of assets
        
        cvar_analysis.append({
            'CVaR Level': f"{confidence*100:.0f}%",
            'Expected Return': portfolio['expected_return'],
            'Volatility': portfolio['volatility'],
            'Sharpe Ratio': portfolio['sharpe_ratio'],
            'Assets Used': num_assets,
            'Max Weight': max_weight,
            'Diversification': diversification,
            'Solve Time': result['solve_time']
        })
    
    cvar_df = pd.DataFrame(cvar_analysis)
    
    print("\nCVaR Sensitivity Analysis Results:")
    print("=" * 40)
    print(cvar_df.round(4).to_string(index=False))
    
    # Visualize CVaR sensitivity
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    confidence_levels = [conf*100 for conf in cvar_results.keys()]
    
    # 1. Risk-Return Trade-off
    axes[0,0].plot(cvar_df['Volatility'], cvar_df['Expected Return'], 'o-', markersize=8)
    axes[0,0].set_xlabel('Volatility')
    axes[0,0].set_ylabel('Expected Return')
    axes[0,0].set_title('CVaR Optimization: Risk-Return Profile')
    axes[0,0].grid(True, alpha=0.3)
    
    # Add confidence level labels
    for i, row in cvar_df.iterrows():
        axes[0,0].annotate(row['CVaR Level'], 
                          (row['Volatility'], row['Expected Return']),
                          xytext=(5, 5), textcoords='offset points')
    
    # 2. Sharpe Ratio Evolution
    axes[0,1].plot(confidence_levels, cvar_df['Sharpe Ratio'], 'o-', color='green', markersize=8)
    axes[0,1].set_xlabel('CVaR Confidence Level (%)')
    axes[0,1].set_ylabel('Sharpe Ratio')
    axes[0,1].set_title('Sharpe Ratio by CVaR Confidence Level')
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. Portfolio Concentration
    axes[1,0].plot(confidence_levels, cvar_df['Max Weight'], 'o-', color='red', markersize=8)
    axes[1,0].set_xlabel('CVaR Confidence Level (%)')
    axes[1,0].set_ylabel('Maximum Weight')
    axes[1,0].set_title('Portfolio Concentration by CVaR Level')
    axes[1,0].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1%}'))
    axes[1,0].grid(True, alpha=0.3)
    
    # 4. Diversification Score
    axes[1,1].plot(confidence_levels, cvar_df['Diversification'], 'o-', color='purple', markersize=8)
    axes[1,1].set_xlabel('CVaR Confidence Level (%)')
    axes[1,1].set_ylabel('Effective Number of Assets')
    axes[1,1].set_title('Diversification by CVaR Level')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ No CVaR results available for analysis")

## 4. Sortino Ratio Optimization

Sortino ratio optimization focuses on maximizing return relative to downside risk only.

In [ ]:
if api_healthy:
    print("📈 Sortino Ratio Optimization Analysis")
    print("=" * 40)
    
    # Compare Sharpe vs Sortino optimization
    objectives_comparison = {
        'Sharpe Ratio': 'maximize_sharpe',
        'Sortino Ratio': 'maximize_sortino'
    }
    
    ratio_results = {}
    
    for name, objective in objectives_comparison.items():
        print(f"\nOptimizing for {name}...")
        result = optimize_portfolio(objective)
        
        if result:
            ratio_results[name] = result
            portfolio = result['portfolio']
            print(f"✅ Success - Expected Return: {portfolio['expected_return']:.4f}")
            print(f"          Volatility: {portfolio['volatility']:.4f}")
            print(f"          Sharpe Ratio: {portfolio['sharpe_ratio']:.3f}")
        
        time.sleep(1)
    
    print(f"\n📊 Completed {len(ratio_results)} ratio optimizations")
else:
    print("❌ API not available - skipping Sortino analysis")
    ratio_results = {}

In [ ]:
# Analyze Sharpe vs Sortino portfolios
if ratio_results and len(ratio_results) >= 2:
    # Create comparison
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Extract portfolio weights
    sharpe_weights = np.array(ratio_results['Sharpe Ratio']['portfolio']['weights'])
    sortino_weights = np.array(ratio_results['Sortino Ratio']['portfolio']['weights'])
    
    # 1. Portfolio Allocation Comparison (Pie Charts)
    for i, (name, weights) in enumerate([('Sharpe Ratio', sharpe_weights), ('Sortino Ratio', sortino_weights)]):
        # Filter out very small weights
        significant_weights = weights > 0.01
        if np.any(significant_weights):
            labels = [SYMBOLS[j] for j in range(len(SYMBOLS)) if significant_weights[j]]
            sizes = weights[significant_weights]
            
            axes[0,i].pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
            axes[0,i].set_title(f'{name} Portfolio Allocation')
    
    # 2. Weight Differences
    weight_diff = sortino_weights - sharpe_weights
    colors = ['red' if x < 0 else 'green' for x in weight_diff]
    
    bars = axes[1,0].bar(SYMBOLS, weight_diff, color=colors, alpha=0.7)
    axes[1,0].set_title('Sortino vs Sharpe Weight Differences')
    axes[1,0].set_ylabel('Weight Difference (Sortino - Sharpe)')
    axes[1,0].tick_params(axis='x', rotation=45)
    axes[1,0].axhline(y=0, color='black', linestyle='-', alpha=0.3)
    axes[1,0].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1%}'))
    axes[1,0].grid(True, alpha=0.3)
    
    # Add value labels
    for bar, value in zip(bars, weight_diff):
        axes[1,0].text(bar.get_x() + bar.get_width()/2, 
                      bar.get_height() + (0.005 if value >= 0 else -0.01),
                      f'{value:.2%}', ha='center', 
                      va='bottom' if value >= 0 else 'top')
    
    # 3. Performance Metrics Comparison
    metrics = ['Expected Return', 'Volatility', 'Sharpe Ratio']
    sharpe_metrics = [ratio_results['Sharpe Ratio']['portfolio'][m.lower().replace(' ', '_')] for m in metrics]
    sortino_metrics = [ratio_results['Sortino Ratio']['portfolio'][m.lower().replace(' ', '_')] for m in metrics]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    axes[1,1].bar(x - width/2, sharpe_metrics, width, label='Sharpe Optimization', alpha=0.7)
    axes[1,1].bar(x + width/2, sortino_metrics, width, label='Sortino Optimization', alpha=0.7)
    
    axes[1,1].set_title('Performance Metrics Comparison')
    axes[1,1].set_ylabel('Metric Value')
    axes[1,1].set_xticks(x)
    axes[1,1].set_xticklabels(metrics)
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate and display additional analysis
    print("\nSharpe vs Sortino Portfolio Analysis:")
    print("=" * 45)
    
    for name, result in ratio_results.items():
        portfolio = result['portfolio']
        weights = np.array(portfolio['weights'])
        
        # Calculate portfolio-level downside metrics
        portfolio_returns = (returns * weights).sum(axis=1)
        downside_returns = portfolio_returns[portfolio_returns < 0]
        portfolio_sortino = (portfolio['expected_return'] - risk_free_rate) / (downside_returns.std() * np.sqrt(252))
        
        print(f"\n{name} Portfolio:")
        print(f"  Expected Return: {portfolio['expected_return']:.4f}")
        print(f"  Volatility: {portfolio['volatility']:.4f}")
        print(f"  Sharpe Ratio: {portfolio['sharpe_ratio']:.3f}")
        print(f"  Calculated Sortino: {portfolio_sortino:.3f}")
        print(f"  Assets with >1% allocation: {np.sum(weights > 0.01)}")
        print(f"  Maximum weight: {np.max(weights):.2%}")
else:
    print("❌ Insufficient data for Sharpe vs Sortino comparison")

## 5. Calmar Ratio Optimization

Calmar ratio optimization focuses on maximizing return relative to maximum drawdown.

In [ ]:
if api_healthy:
    print("📉 Calmar Ratio Optimization Analysis")
    print("=" * 40)
    
    # Test different lookback periods for Calmar optimization
    lookback_periods = [126, 252, 504]  # 6 months, 1 year, 2 years
    calmar_results = {}
    
    for lookback in lookback_periods:
        period_name = f"{lookback//252:.1f}Y" if lookback >= 252 else f"{lookback//21:.0f}M"
        print(f"\nOptimizing Calmar ratio with {period_name} lookback...")
        
        result = optimize_portfolio('maximize_calmar', lookback_periods=lookback)
        
        if result:
            calmar_results[period_name] = result
            portfolio = result['portfolio']
            print(f"✅ Success - Expected Return: {portfolio['expected_return']:.4f}")
            print(f"          Volatility: {portfolio['volatility']:.4f}")
            print(f"          Sharpe Ratio: {portfolio['sharpe_ratio']:.3f}")
        
        time.sleep(1)
    
    # Also compare with Sharpe optimization
    if 'Sharpe Ratio' not in ratio_results:
        print(f"\nOptimizing for Sharpe ratio (reference)...")
        sharpe_result = optimize_portfolio('maximize_sharpe')
        if sharpe_result:
            calmar_results['Sharpe Ref'] = sharpe_result
    
    print(f"\n📊 Completed {len(calmar_results)} Calmar optimizations")
else:
    print("❌ API not available - skipping Calmar analysis")
    calmar_results = {}

In [ ]:
# Analyze Calmar optimization results
if calmar_results:
    # Create Calmar analysis
    calmar_analysis = []
    
    for name, result in calmar_results.items():
        portfolio = result['portfolio']
        weights = np.array(portfolio['weights'])
        
        # Calculate portfolio drawdown characteristics
        portfolio_returns = (returns * weights).sum(axis=1)
        portfolio_prices = (1 + portfolio_returns).cumprod()
        running_max = portfolio_prices.expanding().max()
        drawdowns = (portfolio_prices - running_max) / running_max
        max_dd = drawdowns.min()
        
        # Calculate actual Calmar ratio
        calmar_ratio = portfolio['expected_return'] / abs(max_dd) if max_dd != 0 else 0
        
        calmar_analysis.append({
            'Strategy': name,
            'Expected Return': portfolio['expected_return'],
            'Volatility': portfolio['volatility'],
            'Sharpe Ratio': portfolio['sharpe_ratio'],
            'Max Drawdown': max_dd,
            'Calmar Ratio': calmar_ratio,
            'Assets Used': np.sum(weights > 0.01),
            'Max Weight': np.max(weights)
        })
    
    calmar_df = pd.DataFrame(calmar_analysis)
    
    print("\nCalmar Optimization Results:")
    print("=" * 35)
    display_cols = ['Strategy', 'Expected Return', 'Volatility', 'Sharpe Ratio', 'Max Drawdown', 'Calmar Ratio']
    print(calmar_df[display_cols].round(4).to_string(index=False))
    
    # Visualize Calmar results
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Return vs Drawdown
    axes[0,0].scatter(abs(calmar_df['Max Drawdown']), calmar_df['Expected Return'], 
                     s=100, alpha=0.7, c=calmar_df['Calmar Ratio'], cmap='viridis')
    
    for i, row in calmar_df.iterrows():
        axes[0,0].annotate(row['Strategy'], 
                          (abs(row['Max Drawdown']), row['Expected Return']),
                          xytext=(5, 5), textcoords='offset points')
    
    axes[0,0].set_xlabel('Maximum Drawdown')
    axes[0,0].set_ylabel('Expected Return')
    axes[0,0].set_title('Return vs Maximum Drawdown')
    axes[0,0].xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.1%}'))
    axes[0,0].grid(True, alpha=0.3)
    
    # Add colorbar
    cbar = plt.colorbar(axes[0,0].collections[0], ax=axes[0,0])
    cbar.set_label('Calmar Ratio')
    
    # 2. Calmar Ratio Comparison
    calmar_ratios_finite = calmar_df['Calmar Ratio'][np.isfinite(calmar_df['Calmar Ratio'])]
    strategies_finite = calmar_df['Strategy'][np.isfinite(calmar_df['Calmar Ratio'])]
    
    bars = axes[0,1].bar(strategies_finite, calmar_ratios_finite, alpha=0.7, color='green')
    axes[0,1].set_title('Calmar Ratio Comparison')
    axes[0,1].set_ylabel('Calmar Ratio')
    axes[0,1].tick_params(axis='x', rotation=45)
    axes[0,1].grid(True, alpha=0.3)
    
    # Add value labels
    for bar, value in zip(bars, calmar_ratios_finite):
        axes[0,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                      f'{value:.2f}', ha='center', va='bottom')
    
    # 3. Risk Comparison (Volatility vs Drawdown)
    axes[1,0].scatter(calmar_df['Volatility'], abs(calmar_df['Max Drawdown']), 
                     s=100, alpha=0.7, color='red')
    
    for i, row in calmar_df.iterrows():
        axes[1,0].annotate(row['Strategy'], 
                          (row['Volatility'], abs(row['Max Drawdown'])),
                          xytext=(5, 5), textcoords='offset points')
    
    axes[1,0].set_xlabel('Volatility')
    axes[1,0].set_ylabel('Maximum Drawdown')
    axes[1,0].set_title('Volatility vs Maximum Drawdown')
    axes[1,0].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1%}'))
    axes[1,0].grid(True, alpha=0.3)
    
    # 4. Portfolio Concentration
    axes[1,1].bar(calmar_df['Strategy'], calmar_df['Max Weight'], alpha=0.7, color='purple')
    axes[1,1].set_title('Maximum Weight by Strategy')
    axes[1,1].set_ylabel('Maximum Weight')
    axes[1,1].tick_params(axis='x', rotation=45)
    axes[1,1].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1%}'))
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ No Calmar results available for analysis")

## 6. Comprehensive Objective Comparison

Let's run a comprehensive comparison of all advanced objectives.

In [ ]:
if api_healthy:
    print("🔄 Comprehensive Advanced Objectives Comparison")
    print("=" * 50)
    
    # Define all objectives to compare
    all_objectives = {
        'Sharpe Ratio': {'objective': 'maximize_sharpe'},
        'Min Variance': {'objective': 'minimize_variance'},
        'Max Return': {'objective': 'maximize_return'},
        'CVaR (5%)': {'objective': 'minimize_cvar', 'cvar_confidence': 0.05},
        'Sortino Ratio': {'objective': 'maximize_sortino'},
        'Calmar Ratio': {'objective': 'maximize_calmar', 'lookback_periods': 252}
    }
    
    comprehensive_results = {}
    
    for name, params in all_objectives.items():
        print(f"\nOptimizing {name}...")
        result = optimize_portfolio(**params)
        
        if result:
            comprehensive_results[name] = result
            portfolio = result['portfolio']
            print(f"✅ Success - Return: {portfolio['expected_return']:.4f}, "
                  f"Volatility: {portfolio['volatility']:.4f}, "
                  f"Sharpe: {portfolio['sharpe_ratio']:.3f}")
        else:
            print(f"❌ Failed")
        
        time.sleep(1)
    
    print(f"\n📊 Completed {len(comprehensive_results)} comprehensive optimizations")
else:
    print("❌ API not available - skipping comprehensive analysis")
    comprehensive_results = {}

In [ ]:
# Create comprehensive analysis
if comprehensive_results:
    # Build comprehensive comparison DataFrame
    comp_data = []
    
    for name, result in comprehensive_results.items():
        portfolio = result['portfolio']
        weights = np.array(portfolio['weights'])
        
        # Calculate additional metrics
        portfolio_returns = (returns * weights).sum(axis=1)
        
        # Downside metrics
        downside_returns = portfolio_returns[portfolio_returns < 0]
        downside_vol = downside_returns.std() * np.sqrt(252) if len(downside_returns) > 0 else 0
        sortino = (portfolio['expected_return'] - risk_free_rate) / downside_vol if downside_vol > 0 else 0
        
        # Drawdown metrics
        portfolio_prices = (1 + portfolio_returns).cumprod()
        running_max = portfolio_prices.expanding().max()
        drawdowns = (portfolio_prices - running_max) / running_max
        max_dd = drawdowns.min()
        calmar = portfolio['expected_return'] / abs(max_dd) if max_dd != 0 else 0
        
        # VaR and CVaR (5%)
        var_95 = np.percentile(portfolio_returns, 5) * np.sqrt(252)
        cvar_95 = portfolio_returns[portfolio_returns <= np.percentile(portfolio_returns, 5)].mean() * np.sqrt(252)
        
        comp_data.append({
            'Objective': name,
            'Expected Return': portfolio['expected_return'],
            'Volatility': portfolio['volatility'],
            'Sharpe Ratio': portfolio['sharpe_ratio'],
            'Sortino Ratio': sortino,
            'Calmar Ratio': calmar,
            'Max Drawdown': max_dd,
            'VaR (95%)': var_95,
            'CVaR (95%)': cvar_95,
            'Assets Used': np.sum(weights > 0.01),
            'Max Weight': np.max(weights),
            'Diversification': 1 / np.sum(weights**2),
            'Solve Time': result['solve_time']
        })
    
    comp_df = pd.DataFrame(comp_data)
    
    print("\nComprehensive Objectives Comparison:")
    print("=" * 45)
    key_cols = ['Objective', 'Expected Return', 'Volatility', 'Sharpe Ratio', 'Sortino Ratio', 'Calmar Ratio']
    print(comp_df[key_cols].round(4).to_string(index=False))
    
    print("\nRisk Metrics Comparison:")
    print("=" * 30)
    risk_cols = ['Objective', 'Max Drawdown', 'VaR (95%)', 'CVaR (95%)', 'Assets Used', 'Max Weight']
    print(comp_df[risk_cols].round(4).to_string(index=False))
else:
    print("❌ No comprehensive results available")

In [ ]:
# Create comprehensive visualization
if len(comp_df) > 0:
    fig, axes = plt.subplots(3, 2, figsize=(18, 18))
    
    # 1. Risk-Return Scatter
    scatter = axes[0,0].scatter(comp_df['Volatility'], comp_df['Expected Return'], 
                               s=200, alpha=0.7, c=comp_df['Sharpe Ratio'], cmap='viridis')
    axes[0,0].set_xlabel('Volatility')
    axes[0,0].set_ylabel('Expected Return')
    axes[0,0].set_title('Risk-Return Profile by Objective')
    axes[0,0].grid(True, alpha=0.3)
    
    # Add objective labels
    for i, row in comp_df.iterrows():
        axes[0,0].annotate(row['Objective'], 
                          (row['Volatility'], row['Expected Return']),
                          xytext=(5, 5), textcoords='offset points', fontsize=9)
    
    plt.colorbar(scatter, ax=axes[0,0], label='Sharpe Ratio')
    
    # 2. Risk-Adjusted Performance Ratios
    ratios = ['Sharpe Ratio', 'Sortino Ratio']
    x = np.arange(len(comp_df))
    width = 0.35
    
    axes[0,1].bar(x - width/2, comp_df['Sharpe Ratio'], width, label='Sharpe', alpha=0.7)
    axes[0,1].bar(x + width/2, comp_df['Sortino Ratio'], width, label='Sortino', alpha=0.7)
    axes[0,1].set_title('Risk-Adjusted Performance Comparison')
    axes[0,1].set_ylabel('Ratio Value')
    axes[0,1].set_xticks(x)
    axes[0,1].set_xticklabels(comp_df['Objective'], rotation=45, ha='right')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. Drawdown Analysis
    abs_drawdowns = abs(comp_df['Max Drawdown'])
    bars = axes[1,0].bar(comp_df['Objective'], abs_drawdowns, 
                        color='red', alpha=0.7)
    axes[1,0].set_title('Maximum Drawdown by Objective')
    axes[1,0].set_ylabel('Maximum Drawdown')
    axes[1,0].tick_params(axis='x', rotation=45)
    axes[1,0].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1%}'))
    axes[1,0].grid(True, alpha=0.3)
    
    # 4. Tail Risk Metrics (VaR vs CVaR)
    abs_var = abs(comp_df['VaR (95%)'])
    abs_cvar = abs(comp_df['CVaR (95%)'])
    
    axes[1,1].bar(x - width/2, abs_var, width, label='VaR (95%)', alpha=0.7)
    axes[1,1].bar(x + width/2, abs_cvar, width, label='CVaR (95%)', alpha=0.7)
    axes[1,1].set_title('Tail Risk Metrics Comparison')
    axes[1,1].set_ylabel('Risk Level')
    axes[1,1].set_xticks(x)
    axes[1,1].set_xticklabels(comp_df['Objective'], rotation=45, ha='right')
    axes[1,1].legend()
    axes[1,1].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1%}'))
    axes[1,1].grid(True, alpha=0.3)
    
    # 5. Portfolio Concentration
    axes[2,0].bar(comp_df['Objective'], comp_df['Max Weight'], 
                 color='purple', alpha=0.7)
    axes[2,0].set_title('Portfolio Concentration (Max Weight)')
    axes[2,0].set_ylabel('Maximum Weight')
    axes[2,0].tick_params(axis='x', rotation=45)
    axes[2,0].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1%}'))
    axes[2,0].axhline(y=0.2, color='red', linestyle='--', alpha=0.7, label='20% Limit')
    axes[2,0].legend()
    axes[2,0].grid(True, alpha=0.3)
    
    # 6. Diversification Score
    axes[2,1].bar(comp_df['Objective'], comp_df['Diversification'], 
                 color='green', alpha=0.7)
    axes[2,1].set_title('Portfolio Diversification Score')
    axes[2,1].set_ylabel('Effective Number of Assets')
    axes[2,1].tick_params(axis='x', rotation=45)
    axes[2,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ No data available for comprehensive visualization")

## 7. Key Insights and Recommendations

In [ ]:
if len(comp_df) > 0:
    print("\n" + "="*70)
    print("ADVANCED OBJECTIVES: KEY INSIGHTS & RECOMMENDATIONS")
    print("="*70)
    
    # Generate insights
    insights = []
    
    # Best performers in each category
    best_return = comp_df.loc[comp_df['Expected Return'].idxmax()]
    best_sharpe = comp_df.loc[comp_df['Sharpe Ratio'].idxmax()]
    best_sortino = comp_df.loc[comp_df['Sortino Ratio'].idxmax()]
    lowest_drawdown = comp_df.loc[comp_df['Max Drawdown'].abs().idxmin()]
    lowest_cvar = comp_df.loc[comp_df['CVaR (95%)'].abs().idxmin()]
    
    insights.extend([
        f"🏆 Highest Return: {best_return['Objective']} ({best_return['Expected Return']:.2%})",
        f"📈 Best Risk-Adjusted (Sharpe): {best_sharpe['Objective']} ({best_sharpe['Sharpe Ratio']:.3f})",
        f"📉 Best Downside-Adjusted (Sortino): {best_sortino['Objective']} ({best_sortino['Sortino Ratio']:.3f})",
        f"🛡️ Lowest Drawdown: {lowest_drawdown['Objective']} ({lowest_drawdown['Max Drawdown']:.2%})",
        f"⚠️ Lowest Tail Risk (CVaR): {lowest_cvar['Objective']} ({lowest_cvar['CVaR (95%)']:.2%})"
    ])
    
    # Correlation analysis between metrics
    corr_sharpe_sortino = comp_df['Sharpe Ratio'].corr(comp_df['Sortino Ratio'])
    corr_var_cvar = comp_df['VaR (95%)'].corr(comp_df['CVaR (95%)'])
    
    insights.extend([
        f"🔗 Sharpe-Sortino Correlation: {corr_sharpe_sortino:.3f}",
        f"📊 VaR-CVaR Correlation: {corr_var_cvar:.3f}"
    ])
    
    for i, insight in enumerate(insights, 1):
        print(f"{i}. {insight}")
    
    print("\n" + "="*70)
    print("OBJECTIVE-SPECIFIC RECOMMENDATIONS")
    print("="*70)
    
    recommendations = {
        "CVaR Optimization": [
            "• Best for: Institutions with regulatory capital requirements",
            "• Focus: Tail risk management and worst-case scenario protection",
            "• Trade-off: May sacrifice some upside potential for downside protection",
            "• Confidence level selection: 5% for conservative, 1% for very conservative"
        ],
        "Sortino Ratio Optimization": [
            "• Best for: Investors concerned with downside volatility only",
            "• Focus: Maximizes return per unit of 'bad' volatility",
            "• Advantage: Distinguishes between upside and downside risk",
            "• Use case: Asymmetric return distributions, alternative investments"
        ],
        "Calmar Ratio Optimization": [
            "• Best for: Investors focused on avoiding large losses",
            "• Focus: Return per unit of maximum historical decline",
            "• Strength: Considers worst peak-to-trough experience",
            "• Application: Hedge funds, pension funds, capital preservation"
        ],
        "Implementation Guidelines": [
            "• Conservative portfolios: Prioritize CVaR and Calmar objectives",
            "• Growth portfolios: Balance Sharpe and Sortino optimization",
            "• Institutional mandates: Align objective with regulatory requirements",
            "• Regular rebalancing: Monitor objective-specific metrics over time"
        ]
    }
    
    for category, items in recommendations.items():
        print(f"\n{category}:")
        for item in items:
            print(f"  {item}")
    
    print("\n" + "="*70)
    print("PRACTICAL IMPLEMENTATION NOTES")
    print("="*70)
    
    implementation_notes = [
        "1. 📊 Data Requirements: Advanced objectives need sufficient historical data",
        "2. ⚡ Computational Cost: CVaR and Calmar optimization are more intensive",
        "3. 📈 Market Regime: Different objectives perform better in different conditions",
        "4. 🔄 Rebalancing: Consider transaction costs with complex objectives",
        "5. 🎯 Multi-Objective: Combine objectives for robust portfolio construction",
        "6. 📝 Reporting: Track all risk metrics regardless of primary objective"
    ]
    
    for note in implementation_notes:
        print(note)
    
    # Calculate summary statistics
    print(f"\n📈 Average Expected Return: {comp_df['Expected Return'].mean():.2%}")
    print(f"📊 Average Sharpe Ratio: {comp_df['Sharpe Ratio'].mean():.3f}")
    print(f"📉 Average Max Drawdown: {comp_df['Max Drawdown'].mean():.2%}")
    print(f"⚖️ Risk-Return Efficiency Range: {comp_df['Sharpe Ratio'].min():.3f} to {comp_df['Sharpe Ratio'].max():.3f}")
else:
    print("❌ No comprehensive results available for insights")

## Summary and Conclusions

This comprehensive analysis of advanced portfolio optimization objectives demonstrates the significant impact that different risk measures have on portfolio construction and performance characteristics.

### **Key Findings**

1. **Objective Selection Matters**: Different optimization objectives can lead to substantially different portfolio allocations and risk-return profiles

2. **Risk Measure Impact**: Traditional volatility-based measures (Sharpe) may not capture all aspects of portfolio risk that concern investors

3. **Downside Focus**: Sortino and CVaR objectives provide better downside protection at the potential cost of some upside participation

4. **Drawdown Awareness**: Calmar ratio optimization explicitly considers the worst historical experience, leading to more conservative allocations

### **Practical Applications**

**For Institutional Investors**:
- Use CVaR optimization for regulatory compliance and capital adequacy
- Implement Calmar-based strategies for pension and endowment funds
- Combine multiple objectives for robust risk management

**For Individual Investors**:
- Choose Sortino optimization for asymmetric risk preferences
- Use Calmar optimization when drawdown tolerance is limited
- Consider life-cycle and goal-based objective selection

**For Portfolio Managers**:
- Monitor all risk metrics regardless of primary optimization objective
- Adjust objectives based on market conditions and client preferences
- Use advanced objectives as risk budgeting and allocation tools

### **Technical Achievement**

QuantumEdge's implementation of advanced optimization objectives provides:
- Mathematically rigorous CVaR formulations using CVXPY
- Efficient Sortino ratio optimization with downside deviation constraints
- Robust Calmar ratio optimization with flexible lookback periods
- Seamless integration with existing mean-variance optimization framework

This tutorial demonstrates that modern portfolio optimization extends far beyond traditional mean-variance analysis, offering sophisticated tools for managing different aspects of investment risk. The choice of optimization objective should align with investor preferences, regulatory requirements, and market conditions to achieve optimal risk-adjusted returns.

---

**Continue your QuantumEdge exploration:**
- `05_risk_analysis.ipynb` - Deep dive into comprehensive risk metrics and monitoring
- Review `03_backtesting_strategies.ipynb` to see how these objectives perform historically
- Experiment with different confidence levels, lookback periods, and parameter combinations